In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from poker_embeddings.models.card import CardGNN

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CardGNN().to(device)
model.load_state_dict(torch.load("../model_weights/hand_strength_predictor200.pth", weights_only=True))

<All keys matched successfully>

In [7]:
train_losses = pd.read_csv("../results/hand_rank_predictor_train_loss0.csv")
report = pd.read_csv("../results/hand_rank_predictor_classification_report1.csv")
cm = pd.read_csv("../results/hand_rank_predictor_confusion_matrix1.csv")

In [10]:
train_losses

,train_loss,train_accuracy,epochs
0,1.889245,0.196742,1


In [ ]:
# model.eval()
# with torch.no_grad():
#     card_embeddings = model.card_embedder.weight
#     card_embeddings = model.card_emb_projector(card_embeddings).cpu()
#     card_embeddings_norm = F.normalize(card_embeddings, p=2, dim=1).numpy()

In [ ]:
# from sklearn.manifold import TSNE
# from sklearn.decomposition import PCA

In [ ]:
# tsne = TSNE(n_components=2, perplexity=30, max_iter=300)
# emb_2d = tsne.fit_transform(card_embeddings_norm)

In [12]:
# suit_colors = {
#     0: 'green',    # Clubs
#     1: 'blue',      # Diamonds
#     2: 'red',     # Hearts
#     3: 'black'     # Spades
# }

# colors = [suit_colors[i % 4] for i in range(len(DECK_DICT))]

# # Now plot
# plt.figure(figsize=(6, 6))
# for i, txt in DECK_DICT.items():
#     if i == 52:
#         continue  # skip padding card if you have one
#     plt.scatter(emb_2d[i, 0], emb_2d[i, 1], color=suit_colors[i % 4])
#     plt.annotate(txt, (emb_2d[i, 0], emb_2d[i, 1]), fontsize=8)

# plt.title("Card Embedding Visualization Colored by Suit")
# plt.show()

In [11]:
# rank_colors = plt.cm.get_cmap('tab20', 13)

# colors = [rank_colors(i // 4) for i in range(len(DECK_DICT))]

# plt.figure(figsize=(6, 6))
# for i, txt in DECK_DICT.items():
#     if i == 52:
#         continue
#     plt.scatter(emb_2d[i, 0], emb_2d[i, 1], color=rank_colors(i // 4))
#     plt.annotate(txt, (emb_2d[i, 0], emb_2d[i, 1]), fontsize=8)

# plt.title("Card Embedding Visualization Colored by Rank")
# plt.show()